In [3]:
import ccxt.pro as ccxtpro
import asyncio
import nest_asyncio
import pytz
from datetime import datetime, timedelta
import pandas as pd
from ccxt_lab import whale_1, api_key, api_secret

# Use nest_asyncio to allow async/await code to run in Jupyter Notebook
nest_asyncio.apply()

async def fetch_ohlcv_dataframe(exchange, symbol):
    try:
        current_time_utc = datetime.utcnow() - timedelta(hours=1)
        start_time_str = current_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')
        start_timestamp = exchange.parse8601(start_time_str)

        ohlcv = await exchange.fetch_ohlcv(symbol, timeframe='1m', since=start_timestamp, limit=None)

        df = pd.DataFrame(ohlcv, columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
        df['Timestamp'] = df['Timestamp'] + timedelta(hours=8)
        df['Last5VolumeSum'] = df['Volume'].rolling(window=6, min_periods=1).sum() - df['Volume']

        return df

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

async def watch_ohlcv_and_print(exchange, symbol, whale):
    utc_plus_8 = pytz.timezone('Asia/Shanghai')
    previous_timestamp = None
    entry_time_list = []

    try:
        while True:
            ohlcv = await exchange.watch_ohlcv(symbol, timeframe='1m')
            timestamp_utc = datetime.fromtimestamp(ohlcv[-1][0] / 1000, tz=pytz.utc)
            timestamp_utc8 = timestamp_utc.astimezone(utc_plus_8)
            current_timestamp_str = timestamp_utc8.strftime('%Y-%m-%d %H:%M:%S')
            current_volume = ohlcv[-1][5]

            if current_timestamp_str != previous_timestamp:
                print(f"New timestamp: {current_timestamp_str}")
                df = await fetch_ohlcv_dataframe(exchange, symbol)
                previous_timestamp = current_timestamp_str

            print(f"Current volume: {ohlcv[-1][5]}, Test value: {df['Volume'].iloc[-2]}")

            if df is not None and current_volume > df['Volume'].iloc[-2]:
                if not whale.is_position_open() and current_timestamp_str not in entry_time_list:
                    try:
                        direction = 'buy' if ohlcv[-1][4] > ohlcv[-1][1] else 'sell'
                        place_order = whale.place_first_market_order(direction)
                        print(f"Place order: {place_order}")
                        entry_time_list.append(current_timestamp_str)
                        whale.place_second_take_profit_and_stop_loss_orders()

                    except Exception as e:
                        print(f"An error occurred: {e}")

            print(f"Time: {current_timestamp_str}, Open: {ohlcv[-1][1]}, High: {ohlcv[-1][2]}, Low: {ohlcv[-1][3]}, Close: {ohlcv[-1][4]}, Volume: {ohlcv[-1][5]}, Last 5 volume sum: {df['Last5VolumeSum'].iloc[-1]:.2f}")

    except KeyboardInterrupt:
        pass
    finally:
        await exchange.close()

async def constantly_watch_position_and_orders(exchange, symbol, whale):
    prev_position_exists = False

    try:
        while True:
            positions = await exchange.fetch_positions([symbol])
            current_position_Amt_abs = abs(float(positions[0]['info']['positionAmt']))
            position_exists = (current_position_Amt_abs > 0 or current_position_Amt_abs < 0)

            if position_exists and not prev_position_exists:
                print("Position exists from None")
            
            if not position_exists and prev_position_exists:
                print("No position anymore")
                whale.close_position_and_orders()

            prev_position_exists = position_exists

    except KeyboardInterrupt:
        pass
    finally:
        whale_1.close_position_and_orders()
        await exchange.close()

async def main():
    trading_symbol = 'ETH/USDT'

    exchange = ccxtpro.binance({
        'options': {
            'defaultType': 'future',
            'newUpdates': True
        },
        'api_key': api_key,
        'secret': api_secret
    })
    
    exchange.set_sandbox_mode(True)
    tasks = [
        watch_ohlcv_and_print(exchange, trading_symbol, whale_1),
        constantly_watch_position_and_orders(exchange, trading_symbol, whale_1)
    ]
    await asyncio.gather(*tasks)

api_key = api_key
api_secret = api_secret

asyncio.run(main())


New timestamp: 2023-10-07 17:22:00
Current volume: 18.904, Test value: 60.367
Time: 2023-10-07 17:22:00, Open: 1639.63, High: 1640.01, Low: 1639.57, Close: 1639.71, Volume: 18.904, Last 5 volume sum: 236.80
Current volume: 32.191, Test value: 60.367
Time: 2023-10-07 17:22:00, Open: 1639.63, High: 1640.01, Low: 1639.57, Close: 1639.71, Volume: 32.191, Last 5 volume sum: 236.80
Current volume: 46.671, Test value: 60.367
Time: 2023-10-07 17:22:00, Open: 1639.63, High: 1640.23, Low: 1639.57, Close: 1640.23, Volume: 46.671, Last 5 volume sum: 236.80
Error in closing position: binance amount of ETH/USDT:USDT must be greater than minimum amount precision of 3


KeyboardInterrupt: 

In [14]:
whale_1.place_trailing_stop_order('buy')

An error occurred: binance trailing_stop is not a valid order type for the ETH/USDT market


UnboundLocalError: cannot access local variable 'order' where it is not associated with a value